In [1]:
import CombineML

┌ Info: Precompiling CombineML [f3117721-c1d3-5ff8-9230-e4dd1549a252]
└ @ Base loading.jl:1192
┌ Info: Running `conda install -y scikit-learn` in root environment
└ @ Conda /Users/ppalmes/.julia/packages/Conda/CpuvI/src/Conda.jl:112


Solving environment: ...working... done

# All requested packages already installed.



┌ Warning: RCall.jl: Loading required package: lattice
│ Loading required package: ggplot2
└ @ RCall /Users/ppalmes/.julia/packages/RCall/ffM0W/src/io.jl:113
┌ Warning: RCall.jl: Warning: package ‘e1071’ was built under R version 3.5.2
└ @ RCall /Users/ppalmes/.julia/packages/RCall/ffM0W/src/io.jl:113
┌ Warning: RCall.jl: Loading required package: splines
│ Loading required package: foreach
│ Loaded gam 1.16
│ 
└ @ RCall /Users/ppalmes/.julia/packages/RCall/ffM0W/src/io.jl:113
┌ Warning: RCall.jl: randomForest 4.6-14
│ Type rfNews() to see new features/changes/bug fixes.
│ 
│ Attaching package: ‘randomForest’
│ 
│ The following object is masked from ‘package:ggplot2’:
│ 
│     margin
│ 
└ @ RCall /Users/ppalmes/.julia/packages/RCall/ffM0W/src/io.jl:113
┌ Warning: RCall.jl: 
│ Attaching package: ‘kernlab’
│ 
│ The following object is masked from ‘package:ggplot2’:
│ 
│     alpha
│ 
└ @ RCall /Users/ppalmes/.julia/packages/RCall/ffM0W/src/io.jl:113
┌ Warning: RCall.jl: 
│ Attaching packa

In [2]:
try
  import RDatasets
catch
  using Pkg
  Pkg.add("RDatasets")
  import RDatasets
end

In [3]:
using Distributed
addprocs()

8-element Array{Int64,1}:
 2
 3
 4
 5
 6
 7
 8
 9

In [4]:
@everywhere using Random
@everywhere using Statistics
@everywhere import CombineML
@everywhere import CombineML.Util
@everywhere import CombineML.Transformers
@everywhere import RDatasets

[ Info: Running `conda install -y scikit-learn` in root environment
[ Info: Running `conda install -y scikit-learn` in root environment
[ Info: Running `conda install -y scikit-learn` in root environment
[ Info: Running `conda install -y scikit-learn` in root environment
[ Info: Running `conda install -y scikit-learn` in root environment
[ Info: Running `conda install -y scikit-learn` in root environment
[ Info: Running `conda install -y scikit-learn` in root environment
[ Info: Running `conda install -y scikit-learn` in root environment


      From worker 9:	Collecting package metadata: ...working... done
      From worker 3:	Collecting package metadata: ...working... done
      From worker 7:	Collecting package metadata: ...working... done
      From worker 2:	Collecting package metadata: ...working... done
      From worker 8:	Collecting package metadata: ...working... done
      From worker 6:	Collecting package metadata: ...working... done
      From worker 4:	Collecting package metadata: ...working... done
      From worker 5:	Collecting package metadata: ...working... done
      From worker 3:	Solving environment: ...working... done
      From worker 9:	Solving environment: ...working... done
      From worker 2:	Solving environment: ...working... done
      From worker 4:	Solving environment: ...working... done
      From worker 7:	Solving environment: ...working... done
      From worker 5:	Solving environment: ...working... done
      From worker 8:	Solving environment: ...working... done
      From worker 3:	

┌ Warning: RCall.jl: Loading required package: lattice
│ Loading required package: ggplot2
└ @ RCall ~/.julia/packages/RCall/ffM0W/src/io.jl:113
┌ Warning: RCall.jl: Loading required package: lattice
│ Loading required package: ggplot2
└ @ RCall ~/.julia/packages/RCall/ffM0W/src/io.jl:113
┌ Warning: RCall.jl: Loading required package: lattice
│ Loading required package: ggplot2
└ @ RCall ~/.julia/packages/RCall/ffM0W/src/io.jl:113
┌ Warning: RCall.jl: Loading required package: lattice
│ Loading required package: ggplot2
└ @ RCall ~/.julia/packages/RCall/ffM0W/src/io.jl:113
┌ Warning: RCall.jl: Loading required package: lattice
│ Loading required package: ggplot2
└ @ RCall ~/.julia/packages/RCall/ffM0W/src/io.jl:113
┌ Warning: RCall.jl: Loading required package: lattice
│ Loading required package: ggplot2
└ @ RCall ~/.julia/packages/RCall/ffM0W/src/io.jl:113
┌ Warning: RCall.jl: Loading required package: lattice
│ Loading required package: ggplot2
└ @ RCall ~/.julia/packages/RCall/ffM0W

┌ Warning: RCall.jl: 
│ Attaching package: ‘pls’
│ 
│ The following object is masked from ‘package:caret’:
│ 
│     R2
│ 
│ The following object is masked from ‘package:stats’:
│ 
│     loadings
│ 
└ @ RCall ~/.julia/packages/RCall/ffM0W/src/io.jl:113
┌ Warning: RCall.jl: Warning: package ‘xgboost’ was built under R version 3.5.2
└ @ RCall ~/.julia/packages/RCall/ffM0W/src/io.jl:113
┌ Warning: RCall.jl: Warning: package ‘xgboost’ was built under R version 3.5.2
└ @ RCall ~/.julia/packages/RCall/ffM0W/src/io.jl:113
┌ Warning: RCall.jl: 
│ Attaching package: ‘kernlab’
│ 
│ The following object is masked from ‘package:ggplot2’:
│ 
│     alpha
│ 
└ @ RCall ~/.julia/packages/RCall/ffM0W/src/io.jl:113
┌ Warning: RCall.jl: 
│ Attaching package: ‘pls’
│ 
│ The following object is masked from ‘package:caret’:
│ 
│     R2
│ 
│ The following object is masked from ‘package:stats’:
│ 
│     loadings
│ 
└ @ RCall ~/.julia/packages/RCall/ffM0W/src/io.jl:113
┌ Warning: RCall.jl: Warning: package ‘xgbo

In [5]:
@everywhere CU=CombineML.Util
@everywhere CT=CombineML.Transformers
@everywhere RD=RDatasets

In [6]:
@everywhere function predict(learner)
    dataset = RD.dataset("datasets", "iris")
    features = convert(Matrix,dataset[:, 1:(end-1)])
    labels = convert(Array,dataset[:, end])
    # Split into training and test sets
    (train_ind, test_ind) = CU.holdout(size(features, 1), 0.3)
    # Create pipeline
    pipeline = CT.Pipeline(
      Dict(
        :transformers => [
          CT.OneHotEncoder(), # Encodes nominal features into numeric
          CT.Imputer(), # Imputes NA values
          CT.StandardScaler(), # Standardizes features
          learner # Predicts labels on instances
        ]
      )
    )
    # Train
    CT.fit!(pipeline, features[train_ind, :], labels[train_ind]);
    # Predict
    predictions = CT.transform!(pipeline, features[test_ind, :]);
    # Assess predictions
    result = CU.score(:accuracy, labels[test_ind], predictions)
    return result
end
# Learner with default settings
learner = CT.PrunedTree()
predict(learner)

93.33333333333333

In [8]:
learner = CT.PrunedTree(
  Dict(
    :output => :class,
    :impl_options => Dict(:purity_threshold => 1.0)
  )
)
predict(learner)

95.55555555555556

In [9]:
learner = CT.VoteEnsemble(
  Dict(
    :output => :class,
    :learners => [CT.PrunedTree(), CT.DecisionStumpAdaboost(), CT.RandomForest()]
  )
)
predict(learner)

97.77777777777777

In [10]:
learner = CT.BestLearner(
  Dict(
    :output => :class,
    #:partition_generator => (instances, labels) -> CU.kfold(size(instances, 1), 5),
    #:selection_function => (learner_partition_scores) -> findmax(mean(learner_partition_scores, 2))[2],
    :score_type => Real,
    :learners => [CT.PrunedTree(), CT.DecisionStumpAdaboost(), CT.RandomForest()],
    :learner_options_grid => nothing
  )
)
predict(learner)

95.55555555555556

In [11]:
@everywhere rf = CT.RandomForest(
  Dict(
    :output => :class,
    :impl_options => Dict(
      :num_trees => 100,
      :partial_sampling => 0.7
    )
  )
)
predict(rf)

95.55555555555556

In [12]:
@everywhere ada = CT.DecisionStumpAdaboost(
  Dict(
    :output => :class,
    :impl_options => Dict(:num_iterations => 7)
  )
)
predict(ada)

86.66666666666667

In [13]:
if CombineML.System.LIB_SKL_AVAILABLE
  @everywhere sk_learner = CT.SKLLearner(
   Dict(
        :output => :class,
        #:learner => "BaggingClassifier",
        #:learner => "ExtraTreesClassifier",
        :learner => "GradientBoostingClassifier",
        :impl_options => Dict()
       )
  )
else
  @everywhere sk_learner = CT.Baseline()
end
predict(sk_learner)

97.77777777777777

In [14]:
if CombineML.System.LIB_CRT_AVAILABLE
  @everywhere crt_learner = CT.CRTLearner(
    Dict(
      :output => :class,
      #:learner => "rf",
      :learner => "svmLinear2",
      #:learner => "rpart",
      :impl_options => Dict()
    )
  )
else
  @everywhere crt_learner = CT.Baseline()
end
predict(crt_learner)

97.77777777777777

In [15]:
@everywhere stacklearner = CT.StackEnsemble(Dict(
    :output => :class,
    :learners => [CT.PrunedTree(), CT.RandomForest(),CT.DecisionStumpAdaboost(),rf,crt_learner,sk_learner],
    :stacker =>
       CT.RandomForest(
          Dict(
             :output => :class,
             :impl_options => Dict(
               :num_trees => 20,
               :partial_sampling => 0.7
             )
          )
       ),
    #:stacker_training_proportion => 0.3,
    :keep_original_features => true
))
predict(stacklearner)

93.33333333333333

In [16]:
@everywhere bestlearner = CT.BestLearner(
  Dict(
    :learners => [stacklearner,CT.PrunedTree(), CT.RandomForest(),CT.DecisionStumpAdaboost(),rf,sk_learner,crt_learner],
    :output => :class,
    :score_type => Real,
    :learner_options_grid => nothing
  )
)
predict(bestlearner)

93.33333333333333

In [17]:
@everywhere votelearner = CT.VoteEnsemble(
  Dict(
    :learners => [stacklearner,CT.PrunedTree(), CT.RandomForest(),CT.DecisionStumpAdaboost(),rf,crt_learner,sk_learner],
    :output => :class,
  )
)
predict(votelearner)

95.55555555555556

In [18]:
@everywhere stackstacklearner = CT.StackEnsemble(
  Dict(
    :learners => [stacklearner,votelearner,CT.PrunedTree(), CT.RandomForest(),CT.DecisionStumpAdaboost(),rf,crt_learner,sk_learner],
    #:keep_original_features => false,
    #:stacker_training_proportion => 0.3,
    :stacker =>
       CT.RandomForest(
          Dict(
             :output => :class,
             :impl_options => Dict(
               :num_trees => 20,
               :partial_sampling => 0.7
             )
          )
       )
  )
)
predict(stackstacklearner)

97.77777777777777

In [19]:
using DataFrames
function main(trials)
    learners=Dict(:votelearner=>votelearner,:stacklearner=>stacklearner,
                  :stackstacklearner=>stackstacklearner,
                  :caretlearner=>crt_learner,
                  :scikitlearner=>sk_learner,
                  :juliaprunetree=>CT.PrunedTree(),
                  :juliaadaboost=>CT.DecisionStumpAdaboost(),
                  :baseline=>CT.Baseline(),
                  :juliarandomforest=>CT.RandomForest())
    models=collect(keys(learners))
    ctable=@distributed (vcat) for model in models
        acc=@distributed (vcat) for i=1:trials
            res=predict(learners[model])
            println(model," => ",round(res))
            res
        end
        [model round(mean(acc)) round(std(acc)) length(acc)]
    end
    sorted=sort(DataFrame(ctable),:x2,rev=true)
    rename!(sorted,Dict(:x1=>:model,:x2=>:mean_acc,:x3=>:std_acc,:x4=>:trials))
    return sorted
end
res=main(10)

      From worker 3:	baseline => 29.0
      From worker 3:	baseline => 29.0
      From worker 2:	baseline => 27.0
      From worker 2:	baseline => 27.0
      From worker 4:	scikitlearner => 98.0
      From worker 8:	juliaprunetree => 93.0
      From worker 9:	baseline => 29.0
      From worker 7:	baseline => 22.0
      From worker 8:	juliaprunetree => 96.0
      From worker 2:	juliaprunetree => 93.0
      From worker 2:	juliaprunetree => 100.0
      From worker 3:	juliaprunetree => 96.0
      From worker 9:	juliaprunetree => 96.0
      From worker 7:	juliaprunetree => 96.0
      From worker 8:	baseline => 27.0
      From worker 5:	baseline => 31.0
      From worker 5:	juliaprunetree => 93.0
      From worker 4:	scikitlearner => 93.0
      From worker 4:	baseline => 31.0
      From worker 4:	juliaprunetree => 96.0
      From worker 6:	juliaadaboost => 93.0
      From worker 7:	scikitlearner => 91.0
      From worker 2:	scikitlearner => 98.0
      From worker 3:	scikitlearner => 91.0
   

,model,mean_acc,std_acc,trials
,Any,Any,Any,Any
1,votelearner,96.0,3.0,10
2,stacklearner,95.0,3.0,10
3,scikitlearner,95.0,3.0,10
4,stackstacklearner,95.0,3.0,10
5,juliaprunetree,95.0,2.0,10
6,caretlearner,94.0,3.0,10
7,juliaadaboost,94.0,3.0,10
8,juliarandomforest,94.0,4.0,10
9,baseline,28.0,3.0,10
